In [ ]:
from datetime import datetime
from datetime import timezone
import polars as pl
import os
import plotly.express as px

DATA_DIRECTORY = os.environ.get("DATA_DIRECTORY")

# processed average measurement data
df_1h = pl.scan_parquet(os.path.join(DATA_DIRECTORY, "processed/acropolis_sites", "1h_acropolis_site_data.parquet"))

In [ ]:
df_1h.columns

In [ ]:
df_1h.head().collect()

In [ ]:
start_date = datetime(2024, 2, 29)
df_plot = df_1h.filter(pl.col("creation_timestamp") > start_date.replace(tzinfo=timezone.utc)) \
    .filter(pl.col("Flag") == "O") \
    .collect()

fig = px.line(df_plot, x="creation_timestamp", y="co2", color="site_name")
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='UTC Time (hourly aggregated)',
    title='Continuous Measurements',
)
fig.show()

In [ ]:
def extract_dirunal_hourly(df, site_name:str, start_date=datetime(2024, 2, 1), end_date=datetime.now(), wd:list=[0, 360], ws:int=0, plot:bool=False):
    """_summary_

    Args:
        df (polars): Hourly aggregated dataframe
        site_name (str): Site description short name
        start_date (_type_, optional): Datetime in UTC. Defaults to datetime(2024, 2, 1).
        end_date (_type_, optional): Datetime in UTC. Defaults to datetime.now().
        wd (list, optional): Range of wind direction in °. Defaults to [0, 360].
        ws (int, optional): Wind speed in m/s. Defaults to 0.
        plot (bool, optional): Returns a dataframe if False; Plots if True. Defaults to False.

    Returns:
        df (polars): Returns a dataframe if plot = False
    """
        

    df_extracted = df.filter(pl.col("site_name") == site_name) \
        .filter(pl.col("creation_timestamp") > start_date.replace(tzinfo=timezone.utc)) \
        .filter(pl.col("creation_timestamp") < end_date.replace(tzinfo=timezone.utc)) \
        .filter(pl.col("Flag") == "O") \
        .collect()
        
    if site_name not in ["BLUT_48", "BLUT_85"]:
        df_extracted = df_extracted.filter((pl.col("ws") >= ws)) \
            .filter((pl.col("wd") >= wd[0]) & (pl.col("wd") <= wd[1])) \
   
    df_extracted = df_extracted.with_columns(
        (pl.col("creation_timestamp").dt.hour()).alias("hour of the day")) \
        .group_by(["hour of the day","site_name"]) \
        .agg(pl.col("co2").median()) \
        .sort("hour of the day")
    
    if plot:
        fig = px.line(df_extracted, x="hour of the day", y="co2", markers=True, title=f"{site_name}: CO2 [ppm], Filter for wind between {wd}°")
        fig.show()
    else:
        return df_extracted

In [ ]:
# all sites
date = datetime(2024, 3, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Diurnal Profiles (last 6 months, hourly aggregated)',
)

fig.show()

In [ ]:
# all sites March - April
start_date = datetime(2024, 3, 1)
end_date = datetime(2024, 4, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date, end_date=end_date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Dirunal Profiles (March - April, hourly aggregated)',
)

fig.show()

In [ ]:
# all sites April - May
start_date = datetime(2024, 4, 1)
end_date = datetime(2024, 5, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date, end_date=end_date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Dirunal Profiles (April - May, hourly aggregated)',
)

fig.show()

In [ ]:
# all sites May - June
start_date = datetime(2024, 5, 1)
end_date = datetime(2024, 6, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date, end_date=end_date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Dirunal Profiles (May - June, hourly aggregated)',
)

fig.show()

In [ ]:
# all sites June - July
start_date = datetime(2024, 6, 1)
end_date = datetime(2024, 7, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date, end_date=end_date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Dirunal Profiles (June - July, hourly aggregated)',
)

fig.show()

In [ ]:
# all sites July - August
start_date = datetime(2024, 7, 1)
end_date = datetime(2024, 8, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="SENR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="PASR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="GROR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_85", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="NPLR", start_date=date, end_date=end_date),
    extract_dirunal_hourly(df=df_1h, site_name="HARR", start_date=date, end_date=end_date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.update_layout(
    yaxis_title='CO2 (ppm)',
    xaxis_title='Hour of the Day (UTC)',
    title='Dirunal Profiles (July - August, hourly aggregated)',
)

fig.show()

In [ ]:
# traffic sites filtered for wind

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", wind=[250,290], ws=3),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", wind=[170,270], ws=3),
    extract_dirunal_hourly(df=df_1h, site_name="SENR", wind=[200,250], ws=2),
    
]

df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# south west wind

date = datetime(2024, 7, 9)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", wind=[180,270], ws=2, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", wind=[180,270], ws=1, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SENR", wind=[180,270], ws=1, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", wind=[180,270], ws=1, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BOGR", wind=[180,270], ws=1, start_date=date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# west wind

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", wind=[250,290], ws=2, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", wind=[250,290], ws=3, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", wind=[250,290], ws=3, start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", wind=[250,290], ws=3, start_date=date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# background sites
date = datetime(2024, 3, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="DLRR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="MAIR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TAUR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="FELR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="FINR", wind=[], start_date=date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# inner city
date = datetime(2024, 6, 1)

sites = [
    extract_dirunal_hourly(df=df_1h, site_name="PASR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SENR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="BLUT_48", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="TUMR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SCHR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="RDIR", wind=[], start_date=date),
    extract_dirunal_hourly(df=df_1h, site_name="SWMR", wind=[], start_date=date),
    
]  
df_plot = pl.concat(sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# east wind

sites = ["FINR", "TUMR", "PASR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_dirunal_hourly(df=df_1h, site_name=site, wind=[75,105]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="hour of the day", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
def extract_monthly_site_data(df, site_name:str, wind:list=[], ws=1, start_date=datetime(2024, 2, 1), plot=False):
      
    if wind != []:  
        df_extracted = df.filter(pl.col("site_name") == site_name) \
            .filter(pl.col("creation_timestamp") > start_date.replace(tzinfo=timezone.utc)) \
            .filter((pl.col("wd") > wind[0]) & (pl.col("wd") < wind[1])) \
            .filter((pl.col("ws") > 1)) \
            .filter(pl.col("Flag") == "O") \
            .collect()
    else:
        df_extracted = df.filter(pl.col("site_name") == site_name) \
        .filter(pl.col("creation_timestamp") > start_date.replace(tzinfo=timezone.utc)) \
        .filter(pl.col("Flag") == "O") \
        .collect()
        
    df_extracted = df_extracted.with_columns(
        (pl.col("creation_timestamp").dt.month()).alias("month"))\
        .group_by("month", by=["site_name"]) \
        .agg(pl.col("co2").mean()) \
        .sort("month")
    
    if plot:
        fig = px.line(df_extracted, x="month", y="co2", markers=True, title=f"{site_name}: CO2 [ppm], Filter for wind between {wind}°")
        fig.show()
    else:
        return df_extracted

In [ ]:
# all sites
date = datetime(2024, 3, 1)

sites = [
    extract_monthly_site_data(df=df_1h, site_name="SENR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="DLRR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="TUMR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="RDIR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="SCHR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="FINR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="TAUR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="FELR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="SWMR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="MAIR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="PASR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="GROR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="BLUT_48", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="BLUT_85", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="BOGR", wind=[], start_date=date),
    extract_monthly_site_data(df=df_1h, site_name="HARR", wind=[], start_date=date),  
] 

df_plot = pl.concat(sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# east wind
sites = ["FINR", "TUMR", "PASR", "MAIR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[70,110]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# east wind back ground
sites = ["FINR", "FELR", "NPLR", "TAUR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[70,110]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# west wind downwind
sites = ["FINR", "FELR", "NPLR", "TAUR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[250,290]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# south-east wind back ground
sites = ["MAIR", "DLRR", "GROR", "TAUR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[200,250]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# west wind downwind
sites = ["MAIR", "DLRR", "SWMR", "TUMR", "PASR", "GROR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[80,110]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# inner city
sites = ["BLUT_48", "TUMR", "SWMR", "SCHR", "RDIR", "SENR", "BOGR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# sorounded by green areas
sites = ["MAIR", "DLRR", "TAUR", "FINR", "FELR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()

In [ ]:
# start_date feb
sites = ["TUMR", "TAUR", "MAIR", "DLRR", "FINR", "SENR", "FELR"]
df_sites = []

for site in sites:
    
    df_sites.append(extract_monthly_site_data(df=df_1h, site_name=site, wind=[]))
    
df_plot = pl.concat(df_sites)

fig = px.line(df_plot, x="month", y="co2", color="site_name", markers=True)
fig.show()